## Завдання 1 за допомогою Plotly

In [19]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import VBox, HBox, FloatSlider, Checkbox, Button, Dropdown, Output

### Функція для створення гармоніки з шумом

In [20]:
def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise):
    clean_signal = amplitude * np.sin(2 * np.pi * frequency * time + phase)
    noise = np.random.normal(noise_mean, noise_covariance, size=time.shape) if show_noise else 0
    noisy_signal = clean_signal + noise
    return clean_signal, noisy_signal

### Функція для скидання параметрів

In [21]:
def reset_parameters(button):
    amplitude_slider.value = initial_params['amplitude']
    frequency_slider.value = initial_params['frequency']
    phase_slider.value = initial_params['phase']
    noise_mean_slider.value = initial_params['noise_mean']
    noise_covariance_slider.value = initial_params['noise_covariance']
    show_noise_checkbox.value = initial_params['show_noise']
    signal_color_selection.value = "Blue"
    noise_color_selection.value = "Orange"

### Оновлення графіку при зміні значень слайдерів

In [22]:
def update_graph(change=None):
    clean_signal, noisy_signal = harmonic_with_noise(
        amplitude=amplitude_slider.value,
        frequency=frequency_slider.value,
        phase=phase_slider.value,
        noise_mean=noise_mean_slider.value,
        noise_covariance=noise_covariance_slider.value,
        show_noise=show_noise_checkbox.value,
    )
    signal_color = signal_color_selection.value.lower()
    noise_color = noise_color_selection.value.lower()
    with fig_line.batch_update():
        fig_line.data[0].y = noisy_signal
        fig_line.data[0].line.color = noise_color
        fig_line.data[1].y = clean_signal
        fig_line.data[1].line.color = signal_color
    with fig_dotted.batch_update():
        fig_dotted.data[0].y = noisy_signal
        fig_dotted.data[0].marker.color = noise_color
        fig_dotted.data[1].y = clean_signal
        fig_dotted.data[1].marker.color = signal_color

### Створення додатку

In [23]:
time = np.linspace(0, 10, 1000)

# Початкові параметри
initial_params = {
    "amplitude": 1.0,
    "frequency": 0.5,
    "phase": 0.0,
    "noise_mean": 0.0,
    "noise_covariance": 0.1,
    "show_noise": True,
}

# Генерація початкових сигналів
clean_signal, noisy_signal = harmonic_with_noise(**initial_params)

# Створення графіків для двох вікон
fig_line = go.FigureWidget()
fig_line.add_scatter(x=time, y=noisy_signal, mode='lines', name='Noisy Signal', line=dict(color='orange'))
fig_line.add_scatter(x=time, y=clean_signal, mode='lines', name='Clean Signal', line=dict(color='blue'))
fig_line.update_layout(title="Line Plot", xaxis_title="Time", yaxis_title="Amplitude")

fig_dotted = go.FigureWidget()
fig_dotted.add_scatter(x=time, y=noisy_signal, mode='markers', name='Noisy Signal', marker=dict(color='orange'))
fig_dotted.add_scatter(x=time, y=clean_signal, mode='markers', name='Clean Signal', marker=dict(color='blue'))
fig_dotted.update_layout(title="Dotted Plot", xaxis_title="Time", yaxis_title="Amplitude")

# Віджети для параметрів
amplitude_slider = FloatSlider(value=initial_params['amplitude'], min=0.1, max=2.0, step=0.1, description='Amplitude')
frequency_slider = FloatSlider(value=initial_params['frequency'], min=0.1, max=2.0, step=0.1, description='Frequency')
phase_slider = FloatSlider(value=initial_params['phase'], min=-np.pi, max=np.pi, step=0.1, description='Phase')
noise_mean_slider = FloatSlider(value=initial_params['noise_mean'], min=-1.0, max=1.0, step=0.1, description='Noise Mean')
noise_covariance_slider = FloatSlider(value=initial_params['noise_covariance'], min=0.0, max=1.0, step=0.1, description='Noise Cov')
show_noise_checkbox = Checkbox(value=initial_params['show_noise'], description='Show Noise')
reset_button = Button(description="Reset", button_style="info")
output = Output()

# Додаткові спадні меню для вибору кольорів графіка і шуму
signal_color_selection = Dropdown(
    options=["Orange", "Blue", "Green", "Red"],
    value="Blue",
    description="Signal Color:",
)

noise_color_selection = Dropdown(
    options=["Orange", "Blue", "Green", "Red"],
    value="Orange",
    description="Noise Color:",
)

# Прив'язка подій
amplitude_slider.observe(update_graph, names='value')
frequency_slider.observe(update_graph, names='value')
phase_slider.observe(update_graph, names='value')
noise_mean_slider.observe(update_graph, names='value')
noise_covariance_slider.observe(update_graph, names='value')
show_noise_checkbox.observe(update_graph, names='value')
signal_color_selection.observe(update_graph, names='value')
noise_color_selection.observe(update_graph, names='value')
reset_button.on_click(reset_parameters)

# Відображення інтерфейсу
ui = VBox([
    HBox([amplitude_slider, frequency_slider]),
    HBox([phase_slider, noise_mean_slider]),
    HBox([noise_covariance_slider, show_noise_checkbox]),
    HBox([signal_color_selection, noise_color_selection, reset_button]),
])
VBox([ui, fig_line, fig_dotted])
